In [1]:
import h5py
#import pylab as pl
import numpy as np
#import matplotlib as mpl
import networkx
import ipyvolume as ipv

In [2]:
def split_unique_id(unique_id):
    """Splits the ids assign to the subhalos by the merger tree code by snap number and subfind number """
    subfind_number = int(unique_id % 1e6)
    snap_number = int((unique_id - subfind_number) / 1e6)
    
    return snap_number, subfind_number

def get_main_branch_unique_ids(subtree, node):
    """Gets the unique ids of the subhalos belonging to the main branch of the selected subhalo (node)"""
    mpb = [node, ]
    i = 0
    while True:
        succesors = list(subtree.successors(node))
        if len(succesors) == 0:
            break
        node = succesors[0] # select only the first succesor (main branch)
        mpb.append(node)
        
    return mpb

In [3]:
sim = h5py.File('/data/cielo/simulations/LG1/LG1.hdf5', 'r')
trees = networkx.read_multiline_adjlist('/data/cielo/simulations/LG1/LG1_tree.dat')

In [4]:
def plotItIn3D(fofID, centralID):
    uniqueID = int(127*1e6+fofID)
    stree = networkx.dfs_tree(trees, str(uniqueID))
    mtree = get_main_branch_unique_ids(stree, str(uniqueID))
    
    uniqueC = int(127*1e6+centralID)
    streeC = networkx.dfs_tree(trees, str(uniqueC))
    mtreeC = get_main_branch_unique_ids(streeC, str(uniqueC))
    
    # find all the particleIDs that once belong to the subgroup
    Idlist, zlist = [], []

    galH, cenH = [], []
    
    for mm, mmC in zip(mtree[1:], mtreeC[1:]): # ignore the first one
        snap, idd = split_unique_id(int(mm))
        snapC, iddC = split_unique_id(int(mmC))
        tempOff = sim['SnapNumber_{}/SubGroups/PartType0/Offsets'.format(snap)][idd].astype('int')
        pgal = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snap)][idd]
        if snapC==snap:
            pcen = sim['SnapNumber_{}/SubGroups/SubGroupPos'.format(snapC)][iddC]
        galH.append(pgal)
        cenH.append(pcen)
        if tempOff[0]>= 0 and tempOff[1]>=0:
            tempIds = sim['SnapNumber_{}/PartType0/ParticleIDs'.format(snap)][tempOff[0]:tempOff[1]]
            zt = sim['SnapNumber_{}/Header/Redshift'.format(snap)][()]
            Idlist.extend(list(tempIds))
            zlist.extend([zt]*tempIds.size)

    Idlist, zlist = np.array(Idlist), np.array(zlist)
    Idlist, Idunique = np.unique(Idlist, return_index=True) # remove duplicates
    zlist = zlist[Idunique]

    galH = np.array(galH)
    cenH = np.array(cenH)
    
    # the total list of particles that once belong to a give subgroup, lest see where are today.
    TdayOff = sim['SnapNumber_127/SubGroups/PartType0/Offsets'][fofID].astype('int')
    TdayCoord = sim['SnapNumber_127/PartType0/Coordinates'][TdayOff[0]:TdayOff[1]]

    TdayTotalIDs = sim['SnapNumber_127/PartType0/ParticleIDs'][()]
    
    IndexToday = np.in1d(TdayTotalIDs, Idlist)
    IndexToday_ = np.where(IndexToday)[0]
    
    #remove the ones that are not part of the group today.
    TdayIDs = sim['SnapNumber_127/PartType0/ParticleIDs'][TdayOff[0]:TdayOff[1]]

    IndexNot = np.in1d(Idlist, TdayIDs)
    IdNot = Idlist[np.where(IndexNot==False)[0]]

    IndexT = np.in1d(TdayTotalIDs, IdNot)
    IndexT_ = np.where(IndexT)[0]
    LostCoord = sim['SnapNumber_127/PartType0/Coordinates'][IndexT_]

    # ... and the redshift
    SurvIds_ = TdayTotalIDs[IndexT_]
    iidex = np.in1d(Idlist, SurvIds_)
    iidex_ = np.where(iidex)[0]
    zlist_, Idlist_ = zlist[iidex_], Idlist[iidex_]

    is1 = np.argsort(SurvIds_) # the target
    is2 = np.argsort(Idlist_)

    zorder = np.zeros(IndexT_.size)
    zorder[is1] = zlist_[is2]
    
    #now lets cut at  a certain distance
    distance = 600 # distance in ckpc

    posGal = sim['SnapNumber_127/SubGroups/SubGroupPos'][fofID]
    posCen = sim['SnapNumber_127/SubGroups/SubGroupPos'][centralID]

    dx, dy, dz = LostCoord[:,0]-posGal[0], LostCoord[:,1]-posGal[1], LostCoord[:,2]-posGal[2]
    distToC = np.sqrt(dx**2+dy**2+dz**2)

    icut = np.where(distToC<distance)[0]
    IndexT_near = IndexT_[icut]
    zorder_near = zorder[icut]
    
    NearCoord = sim['SnapNumber_127/PartType0/Coordinates'][IndexT_near]
    
    # ok, lest see velocity and stuff
    velCen = sim['SnapNumber_128/SubGroups/SubGroupVel'][centralID]
    velSat = sim['SnapNumber_128/SubGroups/SubGroupVel'][fofID]

    cenRep = posGal - posCen

    velRep = velSat - velCen
    
    xo, yo, zo = NearCoord[:,0], NearCoord[:,1], NearCoord[:,2]
    xoC, yoC, zoC = xo - posCen[0], yo - posCen[1], zo - posCen[2]
    velX, velY, velZ = velRep[0], velRep[1], velRep[2]

    #cenPx, cenPy, cenPz = galH[:,0] - posCen[0], galH[:,1] - posCen[1], galH[:,2] - posCen[2]
    cenPx, cenPy, cenPz = galH[:,0] - cenH[:,0], galH[:,1] - cenH[:,1], galH[:,2] - cenH[:,2]

    nzbins = 4
    # zlim = np.linspace(zorder_near.min(), zorder_near.max(), nzbins+1)
    zli = np.percentile(zorder, [25, 50, 75, 100])
    zlim = [0.]
    zlim.extend(list(zli))
    zlim = np.array(zlim)

    for i in range(zlim.size):
        zlim[i] = float(int(zlim[i]*10))/10
    
    colourlist = ['blue', 'green', 'orange', 'red']
    
    fig = ipv.figure(width=750, height=750)
    
    for zi, zm, colour in zip(zlim[:-1], zlim[1:], colourlist):
        print(colour,'\t:','{} < z < {}'.format(zi, zm))
        i = np.where((zorder_near>zi) & (zorder_near<zm))
        xoCT ,yoCT, zoCT = xoC[i], yoC[i], zoC[i]
        scatter = ipv.scatter(xoCT, yoCT, zoC, marker='sphere', size=.25, color=colour)
    
    ipv.plot(cenPx, cenPy, cenPz, ls='--', color='black')
    ipv.quiver(np.array([cenRep[0]]), np.array([cenRep[1]]), np.array([cenRep[2]]), np.array([velX]), np.array([velY]), np.array([velZ]), color='black', size=10)
    #ipv.xlabel('x/(ckpc h^{-1})')
    #ipv.ylabel('y/(ckpc h^{-1})')
    #ipv.zlabel('z/(ckpc h^{-1})')
    
    ipv.show()

In [5]:
plotItIn3D(33,32)

blue 	: 0.0 < z < 0.6
green 	: 0.6 < z < 1.4
orange 	: 1.4 < z < 2.3
red 	: 2.3 < z < 8.4


In [6]:
plotItIn3D(35,32)

blue 	: 0.0 < z < 0.1
green 	: 0.1 < z < 1.6
orange 	: 1.6 < z < 1.8
red 	: 1.8 < z < 8.8


In [7]:
plotItIn3D(82,81)

blue 	: 0.0 < z < 0.2
green 	: 0.2 < z < 0.5
orange 	: 0.5 < z < 0.7
red 	: 0.7 < z < 9.8


In [8]:
plotItIn3D(85,81)

blue 	: 0.0 < z < 0.9
green 	: 0.9 < z < 1.3
orange 	: 1.3 < z < 1.5
red 	: 1.5 < z < 8.0
